**Enunciado:** Realizar una herramienta ETL que permita obtener el datamart ventas por internet (Internet Sales) y ventas por revendedores (Reseller Sales) a partir de la base de datos operacional. Documentar las dimensiones que participan y las tablas de hechos (50%). (Octubre 22)

* ¿Cuáles son las dimensiones que participan?

* ¿Cuál es el nivel de granularidad de los datos?

* ¿Cuáles son los indicadores (Medidas) de cada tabla de hechos?

In [26]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import yaml
import os
import numpy as np

In [27]:
config_path = os.path.join(os.getcwd(), "config.yml")

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
    config_oltp = config['OLTP']
    config_olap = config['OLAP']

url_oltp = (f"mssql+pyodbc://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']},{config_oltp['port']}/{config_oltp['dbname']}"
          f"?driver={config_oltp['drivername'].replace(' ', '+')}")

url_olap = (f"mssql+pyodbc://{config_olap['user']}:{config_olap['password']}@{config_olap['host']},{config_olap['port']}/{config_olap['dbname']}"
           f"?driver={config_olap['drivername'].replace(' ', '+')}")
oltp = create_engine(url_oltp)
olap = create_engine(url_olap)

Leer las tablas

In [28]:
def cargaSegura(engine, schema, table):
    inspector = inspect(engine)

    # Obtener columnas
    columnas = [col["name"] for col in inspector.get_columns(table, schema=schema)]
    columnas_problematicas = []

    # Intentar cargar tabla completa
    try:
        return pd.read_sql_table(table_name=table, con=engine, schema=schema)
    except Exception:
        pass

    # Detectar columnas problemáticas
    for col in columnas:
        try:
            pd.read_sql_query(
                f'SELECT TOP 10 "{col}" FROM "{schema}"."{table}"',
                con=engine
            )
        except Exception:
            columnas_problematicas.append(col)


    # Columnas buenas
    columnas_ok = [col for col in columnas if col not in columnas_problematicas]

    # Si no hay columnas válidas
    if not columnas_ok:
        print(f"⚠ La tabla {schema}.{table} no tiene columnas convertibles. Retornando dataframe vacío.")
        return pd.DataFrame()

    # Cargar solo columnas válidas
    query = (
        f'SELECT {", ".join([f"""\"{c}\"""" for c in columnas_ok])} '
        f'FROM "{schema}"."{table}"'
    )

    df = pd.read_sql_query(query, con=engine)
    return df


def extractHumanResources(conection):
    tablas = [
        "Shift", "Department", "Employee", "EmployeeDepartmentHistory", "EmployeePayHistory"
    ]
    humanResources = {}
    for tabla in tablas:
        df = cargaSegura(conection, "HumanResources", tabla)
        humanResources[tabla] = df
        
    return humanResources

def extractPerson(conection):
    tablas = [
        "PersonPhone", "PhoneNumberType", "Address", "AddressType",
        "StateProvince", "BusinessEntity", "BusinessEntityAddress", "BusinessEntityContact",
        "ContactType", "CountryRegion", "EmailAddress", "Password", "Person"
    ]
    person = {}
    for tabla in tablas:
        df = cargaSegura(conection, "Person", tabla)
        person[tabla] = df
        
    return person

def extractProduction(conection):
    tablas = [
        "Product", "ScrapReason", "ProductCategory", "ProductCostHistory", "ProductDescription",
        "ProductDocument", "ProductInventory", "ProductListPriceHistory", "ProductModel",
        "ProductModelIllustration", "ProductModelProductDescriptionCulture", "BillOfMaterials",
        "ProductPhoto", "ProductProductPhoto", "TransactionHistory", "ProductReview",
        "TransactionHistoryArchive", "ProductSubcategory", "UnitMeasure", "WorkOrder",
        "Culture", "WorkOrderRouting", "Document", "Illustration", "Location"
    ]
    production = {}
    for tabla in tablas:
        df = cargaSegura(conection, "Production", tabla)
        production[tabla] = df
        
    return production

def extractPurchasing(conection):
    tablas = [
        "ShipMethod", "ProductVendor", "Vendor", "PurchaseOrderDetail", "PurchaseOrderHeader"
    ]
    purchasing = {}
    for tabla in tablas:
        df = cargaSegura(conection, "Purchasing", tabla)
        purchasing[tabla] = df
        
    return purchasing

def extractSales(conection):
    tablas = [
        "CountryRegionCurrency", "CreditCard", "Currency", "CurrencyRate", "Customer",
        "PersonCreditCard", "SalesOrderDetail", "SalesOrderHeader",
        "SalesOrderHeaderSalesReason", "SalesPerson",
        "SalesPersonQuotaHistory", "SalesReason", "SalesTaxRate",
        "SalesTerritory", "SalesTerritoryHistory", "ShoppingCartItem",
        "SpecialOffer", "SpecialOfferProduct", "Store"
    ]
    sales = {}
    for tabla in tablas:
        df = cargaSegura(conection, "Sales", tabla)
        sales[tabla] = df
        
    return sales

In [29]:
def extractEmployeeHierarchy(engine):
    query = """
    SELECT 
        e.BusinessEntityID AS EmployeeID,
        e.NationalIDNumber AS EmployeeNationalIDAlternateKey,
        e.OrganizationNode.ToString() AS OrgNode,
        m.BusinessEntityID AS ParentEmployeeKey,
        m.NationalIDNumber AS ParentEmployeeNationalIDAlternateKey
    FROM HumanResources.Employee e
    LEFT JOIN HumanResources.Employee m
        ON e.OrganizationNode.GetAncestor(1) = m.OrganizationNode;
    """
    return pd.read_sql_query(query, con=engine)

In [30]:
humanResources =  extractHumanResources(oltp)
person = extractPerson(oltp)
production = extractProduction(oltp)
purchasing = extractPurchasing(oltp) #Funciona
sales = extractSales(oltp) #Funciona

C:\Users\victo\AppData\Local\Temp\ipykernel_2352\3610989036.py:5: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  columnas = [col["name"] for col in inspector.get_columns(table, schema=schema)]
c:\Users\victo\Univalle\SeptimoSemestre\CienciaDatos\ETL-proyecto\my_env\Lib\site-packages\pandas\io\sql.py:1725: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)
c:\Users\victo\Univalle\SeptimoSemestre\CienciaDatos\ETL-proyecto\my_env\Lib\site-packages\pandas\io\sql.py:1725: SAWarning: Did not recognize type 'hierarchyid' of column 'OrganizationNode'
  self.meta.reflect(bind=self.con, only=[table_name], views=True)
C:\Users\victo\AppData\Local\Temp\ipykernel_2352\3610989036.py:5: SAWarning: Did not recognize type 'geography' of column 'SpatialLocation'
  columnas = [col["name"] for col in inspector.get_columns(table, schema=schema)]
c:\Users\victo\Univalle\SeptimoSemestre

In [31]:
sales["Currency"].describe(include='all')

,CurrencyCode,Name,ModifiedDate
count,105,105,105
unique,105,105,NaN
top,AED,Emirati Dirham,NaN
freq,1,1,NaN
mean,NaN,NaN,2008-04-30 00:00:00
min,NaN,NaN,2008-04-30 00:00:00
25%,NaN,NaN,2008-04-30 00:00:00
50%,NaN,NaN,2008-04-30 00:00:00
75%,NaN,NaN,2008-04-30 00:00:00
max,NaN,NaN,2008-04-30 00:00:00


In [32]:
def transformDimCurrency(currency):
    dimCurrency = pd.DataFrame(columns=[
        "CurrencyKey", "CurrencyAlternateKey", "CurrencyName"
    ])
    
    dimCurrency["CurrencyAlternateKey"] = currency["CurrencyCode"] 
    dimCurrency["CurrencyName"] = currency["Name"] 
    dimCurrency["CurrencyKey"] = range(1, len(dimCurrency) + 1)
    
    return dimCurrency

In [33]:
dimCurrency = transformDimCurrency(sales["Currency"])
dimCurrency

,CurrencyKey,CurrencyAlternateKey,CurrencyName
0,1,AED,Emirati Dirham
1,2,AFA,Afghani
2,3,ALL,Lek
3,4,AMD,Armenian Dram
4,5,ANG,Netherlands Antillian Guilder
...,...,...,...
100,101,VEB,Bolivar
101,102,VND,Dong
102,103,XOF,CFA Franc BCEAO
103,104,ZAR,Rand


In [34]:
hierarchy = extractEmployeeHierarchy(oltp)
hierarchy.head()

,EmployeeID,EmployeeNationalIDAlternateKey,OrgNode,ParentEmployeeKey,ParentEmployeeNationalIDAlternateKey
0,1,295847284,None,NaN,None
1,2,245797967,/1/,NaN,None
2,3,509647174,/1/1/,2.0,245797967
3,4,112457891,/1/1/1/,3.0,509647174
4,5,695256908,/1/1/2/,3.0,509647174


In [35]:
def transformDimEmployee(employee, employeePayHistory, employeeDepartmentHistory, department, salesPerson, person, emailAddress, personPhone, hierarchy):
    dimEmployee = pd.DataFrame(columns=[
        "EmployeeKey", "EmployeeNationalIDAlternateKey", "Title", "HireDate", "BirthDate", "LoginID",
        "MaritalStatus", "SalariedFlag", "Gender",
        "VacationHours", "SickLeaveHours", "CurrentFlag", "SalesPersonFlag", "Status"
    ])

    employeePayHistory = (
        employeePayHistory.sort_values("RateChangeDate")
        .groupby("BusinessEntityID")
        .tail(1)
    )
    
    dimEmployee["EmployeeKey"] = employee["BusinessEntityID"]
    dimEmployee["EmployeeNationalIDAlternateKey"] = employee["NationalIDNumber"]
    dimEmployee["Title"] = employee["JobTitle"]
    dimEmployee["HireDate"] = employee["HireDate"]
    dimEmployee["BirthDate"] = employee["BirthDate"]
    dimEmployee["LoginID"] = employee["LoginID"]
    dimEmployee["MaritalStatus"] = employee["MaritalStatus"]
    dimEmployee["SalariedFlag"] = employee["SalariedFlag"].astype(int)
    dimEmployee["Gender"] = employee["Gender"]
    dimEmployee["VacationHours"] = employee["VacationHours"]
    dimEmployee["SickLeaveHours"] = employee["SickLeaveHours"]
    dimEmployee["CurrentFlag"] = employee["CurrentFlag"].astype(int)

    dimEmployee = dimEmployee.merge(
        hierarchy[["EmployeeID", "ParentEmployeeKey", "ParentEmployeeNationalIDAlternateKey"]],
        left_on="EmployeeKey",
        right_on="EmployeeID",
        how="left"
    ).drop(columns=["EmployeeID"])

    dimEmployee = dimEmployee.merge(
        salesPerson[["BusinessEntityID", "TerritoryID"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"]) \
     .rename(columns={"TerritoryID": "SalesTerritoryKey"})
    
    dimEmployee["SalesTerritoryKey"] = dimEmployee["SalesTerritoryKey"].fillna(11)

    dimEmployee = dimEmployee.merge(
        person[["BusinessEntityID", "FirstName", "LastName", "MiddleName", "NameStyle"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"])

    dimEmployee["NameStyle"] = dimEmployee["NameStyle"].astype(int)

    dimEmployee = dimEmployee.merge(
        emailAddress[["BusinessEntityID", "EmailAddress"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"])

    dimEmployee = dimEmployee.merge( 
        personPhone[["BusinessEntityID", "PhoneNumber"]], 
        left_on="EmployeeKey", 
        right_on="BusinessEntityID", 
        how="left" 
    ).drop(columns=["BusinessEntityID"]) \
     .rename(columns={"PhoneNumber": "Phone"})
    
    dimEmployee = dimEmployee.merge(
        employeePayHistory[["BusinessEntityID", "PayFrequency", "Rate"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"]) \
     .rename(columns={"Rate": "BaseRate"})

    
    dimEmployee = dimEmployee.merge(
        employeeDepartmentHistory[["BusinessEntityID", "DepartmentID", "StartDate", "EndDate"]],
        left_on="EmployeeKey",
        right_on="BusinessEntityID",
        how="left"
    ).merge(
        department[["DepartmentID", "Name"]],
        left_on="DepartmentID",
        right_on="DepartmentID",
        how="left"
    ).rename(columns={"Name": "DepartmentName"}).drop(columns=["BusinessEntityID", "DepartmentID"])


    dimEmployee["SalesPersonFlag"] = np.where(
        (dimEmployee["DepartmentName"].str.contains("Sales", na=False)) &
        (dimEmployee["Title"] != "Vice President of Engineering"),
        1,
        0
    )

    dimEmployee["Status"] = np.where(
        dimEmployee["EndDate"].isna(),
        "Current",
        None
    )

    column_order = [
        "EmployeeKey", "ParentEmployeeKey", "EmployeeNationalIDAlternateKey", "ParentEmployeeNationalIDAlternateKey", 
        "SalesTerritoryKey", "FirstName", "LastName", "MiddleName", "NameStyle", "Title", "HireDate", "BirthDate", 
        "LoginID", "EmailAddress", "Phone", "MaritalStatus", "SalariedFlag", "Gender", "PayFrequency", "BaseRate", 
        "VacationHours", "SickLeaveHours", "CurrentFlag", "SalesPersonFlag", "DepartmentName", "StartDate", "EndDate","Status"
    ]

    dimEmployee = dimEmployee[column_order]
    dimEmployee["EmployeeKey"] = range(1, len(dimEmployee) + 1)
    lookup = dimEmployee.set_index("EmployeeNationalIDAlternateKey")["EmployeeKey"].to_dict()
    dimEmployee["ParentEmployeeKey"] = dimEmployee["ParentEmployeeNationalIDAlternateKey"].map(lookup)

    ceo_key = dimEmployee.loc[
        dimEmployee["Title"] == "Chief Executive Officer", "EmployeeKey"
    ].iloc[0]
    dimEmployee.loc[
        dimEmployee["ParentEmployeeKey"].isna() & (dimEmployee["EmployeeKey"] != ceo_key),
        ["ParentEmployeeKey", "ParentEmployeeNationalIDAlternateKey"]
    ] = [
        ceo_key,
        dimEmployee.loc[dimEmployee["EmployeeKey"] == ceo_key, "EmployeeNationalIDAlternateKey"].iloc[0]
    ]
    
    return dimEmployee

In [36]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [37]:
dim = transformDimEmployee(
    humanResources["Employee"],
    humanResources["EmployeePayHistory"],
    humanResources["EmployeeDepartmentHistory"],
    humanResources["Department"],
    sales["SalesPerson"],
    person["Person"],
    person["EmailAddress"],
    person["PersonPhone"],
    hierarchy
)

In [38]:
def transformDimReseller(customer, salesOrderHeader, personPhone, personAddress, personBusinessEntityAddress, demographics):
    dimReseller = pd.DataFrame(columns=[
        "ResellerKey", "GeographyKey", "ResellerAlternateKey", 
         "OrderFrequency", 
        "OrderMonth", "FirstOrderYear", "LastOrderYear",    "MinPaymentType", "MinPaymentAmount", 
         "IDStore"
    ])

    #demographics = utils_etl.extractStoreDemographics(oltp)

    # Este es para usarlo solo para sacar el CustomerID que va a SalesOrderHeader

    customersNoNulos = customer[
        customer["PersonID"].notna() & customer["StoreID"].notna()
    ].copy()  

    # Renombrar CustomerID a CustomerStoreID
    customersNoNulos = customersNoNulos.rename(columns={"CustomerID": "CustomerStoreID"})

    ####

    customer = customer[customer["StoreID"].notna()]


    dimReseller["ResellerKey"] = customer["CustomerID"]
    dimReseller["ResellerAlternateKey"] = customer["AccountNumber"]
    dimReseller["IDStore"] = customer["StoreID"]


    # Datos que se pueden traer desde demographics
    dimReseller = dimReseller.merge(
        demographics[["BusinessEntityID", "ResellerName", "BusinessType", "NumberEmployees", "AnnualSales", "BankName", "AnnualRevenue", "YearOpened", "ProductLine"]],
        left_on="IDStore",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"])

    # Teléfono
    dimReseller = dimReseller.merge(
        personPhone[["BusinessEntityID", "PhoneNumber"]],
        left_on=dimReseller["IDStore"] - 1, # PersonID es StoreID - 1
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"]) \
     .rename(columns={"PhoneNumber": "Phone"})
    
    # Direccion
    dimReseller = dimReseller.merge(
        personBusinessEntityAddress[["BusinessEntityID", "AddressID"]],
        left_on=dimReseller["IDStore"],
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"])

    dimReseller = dimReseller.merge(
        personAddress[["AddressID", "AddressLine1", "AddressLine2"]],
        on="AddressID",
        how="left"
    ).drop(columns=["AddressID"])

    # Tipo de negocio 
    codeBusiness = {"BM": "Value Added Reseller", "BS": "Specialty Bike Shop", "OS": "Warehouse"}
    dimReseller["BusinessType"] = dimReseller["BusinessType"].map(codeBusiness)

    # Orders
    dimReseller = dimReseller.merge(
        customersNoNulos[["CustomerStoreID", "StoreID"]],
        left_on=dimReseller["IDStore"],
        right_on="StoreID",
        how="left"
    )
    

    dimReseller = dimReseller.merge(
        salesOrderHeader[["CustomerID", "OrderDate"]],
        left_on=dimReseller["CustomerStoreID"],
        right_on="CustomerID",
        how="left"
    )

    order_counts = dimReseller.groupby("CustomerStoreID")["OrderDate"].count()
    dimReseller["OrderFrequency"] = dimReseller["CustomerStoreID"].map(order_counts)
    dimReseller["OrderMonth"] = dimReseller["OrderDate"].dt.month
    dimReseller["FirstOrderYear"] = dimReseller.groupby("CustomerStoreID")["OrderDate"].transform("min").dt.year
    dimReseller["LastOrderYear"]  = dimReseller.groupby("CustomerStoreID")["OrderDate"].transform("max").dt.year

    # Frecuency
    



    # Pasar las columnas a int
    cols_int = ["NumberEmployees", "YearOpened",  "OrderMonth", "FirstOrderYear", "LastOrderYear", "OrderFrequency"]

    for c in cols_int:
        dimReseller[c] = dimReseller[c].astype("Int64")
    
    column_order = ["ResellerKey", "GeographyKey", "ResellerAlternateKey", "Phone", "BusinessType", "ResellerName", 
                    "NumberEmployees", "OrderFrequency", "OrderMonth", "FirstOrderYear", "LastOrderYear", 
                    "ProductLine", "AddressLine1", "AddressLine2", "AnnualSales", "BankName", "MinPaymentType", 
                    "MinPaymentAmount", "AnnualRevenue", "YearOpened"]
    dimReseller = dimReseller[column_order]
    dimReseller = dimReseller.drop_duplicates(subset=["ResellerKey"])

    return dimReseller

In [39]:
def extractStoreDemographics(engine):
    query = """
    WITH XMLNAMESPACES (
        'http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/StoreSurvey' AS ss
    )
    SELECT 
        s.BusinessEntityID AS BusinessEntityID,
        s.Name AS ResellerName,
        s.SalesPersonID AS StorePersonID,

        s.Demographics.value('(ss:StoreSurvey/ss:YearOpened)[1]', 'int') AS YearOpened,
        s.Demographics.value('(ss:StoreSurvey/ss:AnnualSales)[1]', 'money') AS AnnualSales,
        s.Demographics.value('(ss:StoreSurvey/ss:AnnualRevenue)[1]', 'money') AS AnnualRevenue,
        s.Demographics.value('(ss:StoreSurvey/ss:NumberEmployees)[1]', 'int') AS NumberEmployees,
        s.Demographics.value('(ss:StoreSurvey/ss:BankName)[1]', 'nvarchar(100)') AS BankName,
        s.Demographics.value('(ss:StoreSurvey/ss:BusinessType)[1]', 'nvarchar(20)') AS BusinessType,
        s.Demographics.value('(ss:StoreSurvey/ss:Specialty)[1]', 'nvarchar(50)') AS ProductLine

    FROM Sales.Store s;
    """
    
    return pd.read_sql_query(query, con=engine)

storeDemographics = extractStoreDemographics(oltp)

In [40]:
dimReseller = transformDimReseller(
    sales["Customer"],
    sales["SalesOrderHeader"],
    person["PersonPhone"],
    person["Address"],
    person["BusinessEntityAddress"],
    storeDemographics
)
dimReseller

,ResellerKey,GeographyKey,ResellerAlternateKey,Phone,BusinessType,ResellerName,NumberEmployees,OrderFrequency,OrderMonth,FirstOrderYear,LastOrderYear,ProductLine,AddressLine1,AddressLine2,AnnualSales,BankName,MinPaymentType,MinPaymentAmount,AnnualRevenue,YearOpened
0,1,NaN,AW00000001,245-555-0173,Value Added Reseller,A Bike Store,2,8,7,2011,2012,Road,2251 Elliot Avenue,None,300000.0,International Bank,NaN,NaN,30000.0,1970
4,2,NaN,AW00000002,170-555-0127,Specialty Bike Shop,Progressive Sports,10,32,6,2012,2014,Mountain,7943 Walnut Ave,None,800000.0,International Security,NaN,NaN,80000.0,1972
20,3,NaN,AW00000003,279-555-0130,Warehouse,Advanced Bike Components,40,24,8,2011,2014,Road,12345 Sterling Avenue,None,1500000.0,Primary International,NaN,NaN,150000.0,1974
32,4,NaN,AW00000004,710-555-0173,Value Added Reseller,Modular Cycle Systems,5,32,5,2012,2014,Road,800 Interchange Blvd.,Suite 2501,300000.0,United Security,NaN,NaN,30000.0,1976
48,5,NaN,AW00000005,828-555-0186,Specialty Bike Shop,Metropolitan Sports Supply,13,16,7,2012,2014,Road,482505 Warm Springs Blvd.,None,800000.0,Primary Bank & Reserve,NaN,NaN,80000.0,1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7776,30114,NaN,AW00030114,1 (11) 500 555-0116,Value Added Reseller,Recreation Toy Store,11,16,5,2012,2014,Mountain,"39, avenue des Laurentides",None,800000.0,International Bank,NaN,NaN,80000.0,1988
7784,30115,NaN,AW00030115,155-555-0140,Value Added Reseller,Retreat Inn,8,16,6,2012,2014,Road,Suite 2502 410 Albert Street,None,300000.0,Primary Bank & Reserve,NaN,NaN,30000.0,1982
7792,30116,NaN,AW00030116,433-555-0168,Value Added Reseller,Technical Parts Manufacturing,5,8,5,2013,2014,Touring,Ontario Mills,None,300000.0,International Security,NaN,NaN,30000.0,1976
7796,30117,NaN,AW00030117,560-555-0171,Value Added Reseller,Totes & Baskets Company,2,24,7,2011,2014,Road,72540 Blanco Rd.,None,300000.0,Guardian Bank,NaN,NaN,30000.0,1970


In [45]:
def transformFactResellerSales(product, salesOrderDetail, salesOrderHeader, dimCurrency, currencyRate, dimReseller, customer, salesPerson, dimEmployee, store, employee):
    salesOrderDetail = salesOrderDetail.copy()
    salesOrderDetail["SalesOrderLineNumber"] = (
        salesOrderDetail.groupby("SalesOrderID").cumcount() + 1
    )

    # Start building factResellerSales from salesOrderDetail
    factResellerSales = salesOrderDetail[["ProductID", "SalesOrderID", "SpecialOfferID", 
                                            "SalesOrderLineNumber", "OrderQty", "UnitPrice", 
                                            "UnitPriceDiscount", "LineTotal", "CarrierTrackingNumber"]].rename(
        columns={"ProductID": "ProductKey"}
    )


    # Now merge with salesOrderHeader
    factResellerSales = factResellerSales.merge(
        salesOrderHeader[["SalesOrderID", "SalesOrderNumber", "RevisionNumber", "OrderDate", 
                            "DueDate", "ShipDate", "CustomerID", "TerritoryID", 
                            "Freight", "CurrencyRateID", "TaxAmt"]],
        on="SalesOrderID",
        how="left"
    ).rename(columns={
        "SpecialOfferID": "PromotionKey", 
        "OrderQty": "OrderQuantity", 
        "UnitPriceDiscount": "UnitPriceDiscountPct", 
        "TerritoryID": "SalesTerritoryKey", 
        "LineTotal": "SalesAmount"
    }).drop(columns=["SalesOrderID"])

    
    factResellerSales = factResellerSales.merge(
        dimReseller[["ResellerKey", "ResellerAlternateKey"]],
        left_on="CustomerID",
        right_on="ResellerKey",
        how="left"
    ).drop(columns=["CustomerID"])

    

    factResellerSales = factResellerSales[factResellerSales["ResellerKey"].notna()]
    
    factResellerSales = factResellerSales.merge(
        product[["ProductID", "StandardCost"]],
        left_on="ProductKey",
        right_on="ProductID",
        how="left"
    ).rename(columns={"StandardCost": "ProductStandardCost"}) \
    .drop(columns=["ProductID"])
    
    factResellerSales = factResellerSales.merge(
        currencyRate[["CurrencyRateID", "ToCurrencyCode"]],
        on="CurrencyRateID",
        how="left"
    ).drop(columns=["CurrencyRateID"]).merge(
        dimCurrency[["CurrencyAlternateKey", "CurrencyKey"]],
        left_on="ToCurrencyCode",
        right_on="CurrencyAlternateKey",
        how="left"
    ).drop(columns=["CurrencyAlternateKey", "ToCurrencyCode"])
    
    factResellerSales = factResellerSales.merge(
        customer[["CustomerID", "StoreID"]],
        left_on="ResellerKey",
        right_on="CustomerID",
        how="left"
    ).drop(columns=["CustomerID"]).merge(
        store[["SalesPersonID", "BusinessEntityID"]],
        left_on="StoreID",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"]).merge(
        salesPerson[["BusinessEntityID"]],
        left_on="SalesPersonID",
        right_on="BusinessEntityID",
        how="left"
    ).drop(columns=["SalesPersonID"]).merge(
        employee[["BusinessEntityID", "NationalIDNumber"]],
        on="BusinessEntityID",
        how="left"
    ).drop(columns=["BusinessEntityID"]).merge(
        dimEmployee[["EmployeeKey", "EmployeeNationalIDAlternateKey"]],
        left_on="NationalIDNumber",
        right_on="EmployeeNationalIDAlternateKey",
        how="left"
    ).drop(columns=["NationalIDNumber", "EmployeeNationalIDAlternateKey"])
    

    def transforma_date(date):
        if pd.isna(date):
            return None
        return int(date.strftime("%Y%m%d"))
    
    factResellerSales["OrderDateKey"] = factResellerSales["OrderDate"].apply(transforma_date).astype("Int64")
    factResellerSales["DueDateKey"] = factResellerSales["DueDate"].apply(transforma_date).astype("Int64")
    factResellerSales["ShipDateKey"] = factResellerSales["ShipDate"].apply(transforma_date).astype("Int64")
    
    factResellerSales["ExtendedAmount"] = factResellerSales["UnitPrice"] * factResellerSales["OrderQuantity"]
    factResellerSales["DiscountAmount"] = factResellerSales["ExtendedAmount"] * factResellerSales["UnitPriceDiscountPct"]
    factResellerSales["TotalProductCost"] = factResellerSales["ProductStandardCost"] * factResellerSales["OrderQuantity"]

    

    column_order = ["ProductKey", "OrderDateKey", "DueDateKey", "ShipDateKey", "ResellerKey", "EmployeeKey","PromotionKey", "CurrencyKey",
        "SalesTerritoryKey", "SalesOrderNumber", "SalesOrderLineNumber", "RevisionNumber", "OrderQuantity", 
        "UnitPrice", "ExtendedAmount", "UnitPriceDiscountPct", "DiscountAmount", "ProductStandardCost", "TotalProductCost",
        "SalesAmount", "TaxAmt", "Freight", "CarrierTrackingNumber", "OrderDate", "DueDate", "ShipDate"]
    
    factResellerSales = factResellerSales[column_order]
    
    return factResellerSales

In [46]:
fact = transformFactResellerSales(
    production["Product"],
    sales["SalesOrderDetail"],
    sales["SalesOrderHeader"],
    dimCurrency,
    sales["CurrencyRate"],
    dimReseller,
    sales["Customer"], 
    sales["SalesPerson"], 
    dim, 
    sales["Store"], 
    humanResources["Employee"]
    
)

fact
#fact[fact["SalesOrderNumber"] == "SO43659"]
#dimCustomer[dimCustomer["CustomerKey"] == 11146]

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,ResellerKey,EmployeeKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,RevisionNumber,OrderQuantity,UnitPrice,ExtendedAmount,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,OrderDate,DueDate,ShipDate
0,776,20110531,20110612,20110607,29825.0,285,1,NaN,5,SO43659,1,8,1,2024.994,2024.994,0.0,0.0000,1898.0944,1898.0944,2024.9940,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
1,777,20110531,20110612,20110607,29825.0,285,1,NaN,5,SO43659,2,8,3,2024.994,6074.982,0.0,0.0000,1898.0944,5694.2832,6074.9820,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
2,778,20110531,20110612,20110607,29825.0,285,1,NaN,5,SO43659,3,8,1,2024.994,2024.994,0.0,0.0000,1898.0944,1898.0944,2024.9940,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
3,771,20110531,20110612,20110607,29825.0,285,1,NaN,5,SO43659,4,8,1,2039.994,2039.994,0.0,0.0000,1912.1544,1912.1544,2039.9940,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
4,772,20110531,20110612,20110607,29825.0,285,1,NaN,5,SO43659,5,8,1,2039.994,2039.994,0.0,0.0000,1912.1544,1912.1544,2039.9940,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60914,920,20140501,20140513,20140508,30046.0,283,1,NaN,4,SO71952,37,8,2,158.430,316.860,0.0,0.0000,144.5938,289.1876,316.8600,6573.0031,2054.0635,9490-4552-81,2014-05-01,2014-05-13,2014-05-08
60915,743,20140501,20140513,20140508,30046.0,283,1,NaN,4,SO71952,38,8,1,809.760,809.760,0.0,0.0000,739.0410,739.0410,809.7600,6573.0031,2054.0635,9490-4552-81,2014-05-01,2014-05-13,2014-05-08
60916,742,20140501,20140513,20140508,30046.0,283,1,NaN,4,SO71952,39,8,4,818.700,3274.800,0.0,0.0000,747.2002,2988.8008,3274.8000,6573.0031,2054.0635,9490-4552-81,2014-05-01,2014-05-13,2014-05-08
60917,994,20140501,20140513,20140508,30046.0,283,1,NaN,4,SO71952,40,8,3,32.394,97.182,0.0,0.0000,23.9716,71.9148,97.1820,6573.0031,2054.0635,9490-4552-81,2014-05-01,2014-05-13,2014-05-08


In [ ]:
import xml.etree.ElementTree as ET
def extraerDemografia(df, xml_col):
    data = []
    
    for xml_str in df[xml_col]:
        try:
            root = ET.fromstring(xml_str)
            row = {child.tag.split('}')[1]: child.text for child in root}
            data.append(row)
        except ET.ParseError:
            # En caso de que haya XML mal formado
            data.append({})
    
    df_parsed = pd.DataFrame(data)
    
    # Columnas numéricas conocidas
    numeric_cols = [
        'TotalPurchaseYTD', 'TotalChildren', 'NumberChildrenAtHome',
        'NumberCarsOwned', 'HomeOwnerFlag'
    ]
    
    for col in numeric_cols:
        if col in df_parsed.columns:
            df_parsed[col] = pd.to_numeric(df_parsed[col], errors='coerce')
    
    # Columnas de fecha conocidas
    date_cols = ['BirthDate', 'DateFirstPurchase']
    
    for col in date_cols:
        if col in df_parsed.columns:
            df_parsed[col] = df_parsed[col].str.replace('Z','', regex=False)  # quitar la Z
            df_parsed[col] = pd.to_datetime(df_parsed[col], errors='coerce', format='%Y-%m-%d')
    
    return df_parsed

In [ ]:
def transformDimCustomer(person, sales):
    #Tipos IN = Individual Customer
    dimCustomer = person["Person"][person["Person"]["PersonType"] == 'IN'].copy()
    dimCustomer = dimCustomer.drop(columns=[
        'PersonType', 'EmailPromotion', 'AdditionalContactInfo', 'ModifiedDate', 'rowguid'
    ])
    
    demografia = extraerDemografia(dimCustomer,"Demographics").drop(columns=[
        'TotalPurchaseYTD'
    ])
    demografia = demografia.rename(columns={
        'Education': 'EnglishEducation',
        'Occupation': 'EnglishOccupation',
    })
    
    #Añadir español y francés
    education_map = {
        "Bachelors": {"Spanish": "Licenciatura", "French": "Bac + 4"},
        "Graduate Degree": {"Spanish": "Estudios de postgrado", "French": "Bac + 3"},
        "High School": {"Spanish": "Educación secundaria", "French": "Bac + 2"},
        "Partial College": {"Spanish": "Estudios universitarios (en curso)", "French": "Baccalauréat"},
        "Partial High School": {"Spanish": "Educación secundaria (en curso)", "French": "Niveau bac"}
    }
    occupation_map = {
        "Clerical": {"Spanish": "Administrativo", "French": "Employé"},
        "Management": {"Spanish": "Gestión", "French": "Direction"},
        "Manual": {"Spanish": "Obrero", "French": "Ouvrier"},
        "Professional": {"Spanish": "Profesional", "French": "Cadre"},
        "Skilled Manual": {"Spanish": "Obrero especializado", "French": "Technicien"}
    }
    demografia["EnglishEducation"] = demografia["EnglishEducation"].str.strip()
    demografia["EnglishOccupation"] = demografia["EnglishOccupation"].str.strip()

    demografia["SpanishEducation"] = demografia["EnglishEducation"].map(lambda x: education_map[x]["Spanish"])
    demografia["FrenchEducation"] = demografia["EnglishEducation"].map(lambda x: education_map[x]["French"])
    
    demografia["SpanishOccupation"] = demografia["EnglishOccupation"].map(lambda x: occupation_map[x]["Spanish"])
    demografia["FrenchOccupation"] = demografia["EnglishOccupation"].map(lambda x: occupation_map[x]["French"])
    
    dimCustomer = pd.concat([dimCustomer, demografia], axis=1)
    
    businessEntityAddress = person["BusinessEntityAddress"]
    direccion = person["Address"].drop(columns=['rowguid'])
    customer = sales["Customer"].drop(columns=['rowguid'])
    phone = person["PersonPhone"].drop(columns=['ModifiedDate', 'PhoneNumberTypeID'])
    email = person["EmailAddress"].drop(columns=['EmailAddressID', 'rowguid', 'ModifiedDate'])
    
    dimCustomer = dimCustomer.merge(customer[customer['PersonID'].notna()], left_on='BusinessEntityID', right_on='PersonID', how='inner').drop(columns=['PersonID'])
    dimCustomer = dimCustomer.merge(businessEntityAddress, on='BusinessEntityID', how='left')
    dimCustomer = dimCustomer.merge(direccion, on='AddressID', how='left')
    dimCustomer = dimCustomer.merge(phone, on='BusinessEntityID', how='left')
    dimCustomer = dimCustomer.merge(email, on='BusinessEntityID', how='left')

    dimCustomer['CustomerKey'] = range(11000, 11000 + len(dimCustomer))
    dimCustomer = dimCustomer.merge(
        customer[customer['PersonID'].notna()][['PersonID', 'AccountNumber']],
        left_on='BusinessEntityID',
        right_on='PersonID',
        how='left'
    ).rename(columns={'AccountNumber_y': 'CustomerAlternateKey'})

    
    dimCustomer = dimCustomer.drop(columns=['BusinessEntityID', 'Demographics', 'CustomerID', 'StoreID', 'TerritoryID', 
       'ModifiedDate_x', 'AddressID', 'AddressTypeID', 'PersonID',
       'rowguid', 'ModifiedDate_y', 'City', 'AccountNumber_x',
       'StateProvinceID', 'ModifiedDate',       
    ])
    
    return dimCustomer

In [ ]:
dimCustomer = transformDimCustomer(
    person,
    sales)

In [ ]:
def transformFactInternetSales(product, salesOrderDetail, salesOrderHeader, customer, dimCustomer, dimCurrency, currencyRate, stateProvince, salesTaxRate):
    salesOrderDetail = salesOrderDetail.copy()
    salesOrderDetail["SalesOrderLineNumber"] = (
        salesOrderDetail.groupby("SalesOrderID").cumcount() + 1
    )

    # Start building factInternetSales from salesOrderDetail
    factInternetSales = salesOrderDetail[["ProductID", "SalesOrderID", "SpecialOfferID", 
                                            "SalesOrderLineNumber", "OrderQty", "UnitPrice", 
                                            "UnitPriceDiscount", "LineTotal", "CarrierTrackingNumber"]].rename(
        columns={"ProductID": "ProductKey"}
    )

    # Now merge with salesOrderHeader
    factInternetSales = factInternetSales.merge(
        salesOrderHeader[["SalesOrderID", "SalesOrderNumber", "RevisionNumber", "OrderDate", 
                            "DueDate", "ShipDate", "CustomerID", "TerritoryID", 
                            "Freight", "CurrencyRateID", "TaxAmt"]],
        on="SalesOrderID",
        how="left"
    ).rename(columns={
        "SpecialOfferID": "PromotionKey", 
        "OrderQty": "OrderQuantity", 
        "UnitPriceDiscount": "UnitPriceDiscountPct", 
        "TerritoryID": "SalesTerritoryKey", 
        "LineTotal": "SalesAmount"
    }).drop(columns=["SalesOrderID"])
    
    # Rest of your code remains the same...
    factInternetSales = factInternetSales.merge(
        customer[["CustomerID", "AccountNumber"]],
        on="CustomerID",
        how="left"
    ).drop(columns=["CustomerID"]).merge(
        dimCustomer[["CustomerAlternateKey", "CustomerKey"]],
        left_on="AccountNumber",
        right_on="CustomerAlternateKey",
        how="left"
    ).drop(columns=["CustomerAlternateKey", "AccountNumber"])
    
    factInternetSales = factInternetSales.merge(
        product[["ProductID", "StandardCost"]],
        left_on="ProductKey",
        right_on="ProductID",
        how="left"
    ).rename(columns={"StandardCost": "ProductStandardCost"}) \
    .drop(columns=["ProductID"])
    
    factInternetSales = factInternetSales.merge(
        currencyRate[["CurrencyRateID", "ToCurrencyCode"]],
        on="CurrencyRateID",
        how="left"
    ).drop(columns=["CurrencyRateID"]).merge(
        dimCurrency[["CurrencyAlternateKey", "CurrencyKey"]],
        left_on="ToCurrencyCode",
        right_on="CurrencyAlternateKey",
        how="left"
    ).drop(columns=["CurrencyAlternateKey", "ToCurrencyCode"])

    

    def transforma_date(date):
        if pd.isna(date):
            return None
        return int(date.strftime("%Y%m%d"))
    
    factInternetSales["OrderDateKey"] = factInternetSales["OrderDate"].apply(transforma_date).astype("Int64")
    factInternetSales["DueDateKey"] = factInternetSales["DueDate"].apply(transforma_date).astype("Int64")
    factInternetSales["ShipDateKey"] = factInternetSales["ShipDate"].apply(transforma_date).astype("Int64")
    
    factInternetSales["ExtendedAmount"] = factInternetSales["UnitPrice"] * factInternetSales["OrderQuantity"]
    factInternetSales["DiscountAmount"] = factInternetSales["ExtendedAmount"] * factInternetSales["UnitPriceDiscountPct"]
    factInternetSales["TotalProductCost"] = factInternetSales["ProductStandardCost"] * factInternetSales["OrderQuantity"]

    

    column_order = ["ProductKey", "OrderDateKey", "DueDateKey", "ShipDateKey", "CustomerKey", "PromotionKey", "CurrencyKey",
        "SalesTerritoryKey", "SalesOrderNumber", "SalesOrderLineNumber", "RevisionNumber", "OrderQuantity", 
        "UnitPrice", "ExtendedAmount", "UnitPriceDiscountPct", "DiscountAmount", "ProductStandardCost", "TotalProductCost",
        "SalesAmount", "TaxAmt", "Freight", "CarrierTrackingNumber", "OrderDate", "DueDate", "ShipDate"]
    
    factInternetSales = factInternetSales[column_order]
    
    return factInternetSales

In [ ]:
dim = transformFactInternetSales(
    production["Product"],
    sales["SalesOrderDetail"],
    sales["SalesOrderHeader"],
    sales["Customer"],
    dimCustomer,
    dimCurrency,
    sales["CurrencyRate"],
    person["StateProvince"],
    sales["SalesTaxRate"]
    )
dim[dim["SalesOrderNumber"] == "SO43659"]

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,RevisionNumber,OrderQuantity,UnitPrice,ExtendedAmount,UnitPriceDiscountPct,DiscountAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,OrderDate,DueDate,ShipDate
0,776,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,1,8,1,2024.9940,2024.9940,0.0,0.0,1898.0944,1898.0944,2024.9940,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
1,777,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,2,8,3,2024.9940,6074.9820,0.0,0.0,1898.0944,5694.2832,6074.9820,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
2,778,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,3,8,1,2024.9940,2024.9940,0.0,0.0,1898.0944,1898.0944,2024.9940,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
3,771,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,4,8,1,2039.9940,2039.9940,0.0,0.0,1912.1544,1912.1544,2039.9940,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
4,772,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,5,8,1,2039.9940,2039.9940,0.0,0.0,1912.1544,1912.1544,2039.9940,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
5,773,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,6,8,2,2039.9940,4079.9880,0.0,0.0,1912.1544,3824.3088,4079.9880,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
6,774,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,7,8,1,2039.9940,2039.9940,0.0,0.0,1912.1544,1912.1544,2039.9940,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
7,714,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,8,8,3,28.8404,86.5212,0.0,0.0,38.4923,115.4769,86.5212,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
8,716,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,9,8,1,28.8404,28.8404,0.0,0.0,38.4923,38.4923,28.8404,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
9,709,20110531,20110612,20110607,NaN,1,NaN,5,SO43659,10,8,6,5.7000,34.2000,0.0,0.0,3.3963,20.3778,34.2000,1971.5149,616.0984,4911-403C-98,2011-05-31,2011-06-12,2011-06-07
